In [104]:
!pip install python-rake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install summa

In [287]:
import re
import os
from nltk import pos_tag
from tqdm.auto import tqdm
import RAKE
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import TextBlob, Word
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from summa import keywords
import spacy
nlp = spacy.load('en_core_web_sm')
from sklearn.metrics import f1_score, precision_score,recall_score
from sklearn.preprocessing import MultiLabelBinarizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## 1-2. Описание корпуса, загрузка данных, ручная разметка ключевых слов

Я использовала корпус [Inspec](https://github.com/LIAAD/KeywordExtractor-Datasets/blob/master/datasets/Inspec.zip)


Description: Inspec consists of 2,000 abstracts of scientific journal papers from Computer Science collected between the years 1998 and 2002. Each document has two sets of keywords assigned: the controlled keywords, which are manually controlled assigned keywords that appear in the Inspec thesaurus but may not appear in the document, and the uncontrolled keywords which are freely assigned by the editors, i.e., are not restricted to the thesaurus or to the document. In our repository, we consider a union of both sets as the ground-truth.

Для задания я взяла 27 текстов - abstracts из научных журналов. Средний размер каждого текста 128 токенов, поэтому для получения корпуса из 3-5 тысяч токенов я использовала 30 текстов. Данные в корпусе представлены в виде набора файлов с расширениями .key и .txt

Для каждого abstract, то есть для каждого файла .txt есть свой файл .key с таким же названием.

Помимо этого, для каждого .key файла после эталонной разметки я составила свою.

Пример:

эталонное_слово_1

эталонное_слово_2

эталонное_слово_3

эталонное_слово_4

##

мое_слово_1

мое_слово_2

мое_слово_3

In [208]:
txt_files = []
key_files = []
def go_through_files(curr_dir):  # функция для сбора всех файлов в один список
    for root, dirs, files in os.walk(curr_dir):
        for name in files:
            if re.match(r'.*\.key', name):
                key_files.append(os.path.join(root, name))
            if re.match(r'.*\.txt', name):
                txt_files.append(os.path.join(root, name))
    return sorted(txt_files), sorted(key_files)


In [209]:
txt_files, key_files = go_through_files('corpus')

In [210]:
txt_files

['corpus/11.txt',
 'corpus/12.txt',
 'corpus/13.txt',
 'corpus/14.txt',
 'corpus/15.txt',
 'corpus/16.txt',
 'corpus/17.txt',
 'corpus/18.txt',
 'corpus/19.txt',
 'corpus/2.txt',
 'corpus/20.txt',
 'corpus/21.txt',
 'corpus/22.txt',
 'corpus/23.txt',
 'corpus/24.txt',
 'corpus/25.txt',
 'corpus/26.txt',
 'corpus/27.txt',
 'corpus/28.txt',
 'corpus/29.txt',
 'corpus/3.txt',
 'corpus/30.txt',
 'corpus/4.txt',
 'corpus/6.txt',
 'corpus/7.txt',
 'corpus/8.txt',
 'corpus/9.txt']

In [211]:
key_files

['corpus/11.key',
 'corpus/12.key',
 'corpus/13.key',
 'corpus/14.key',
 'corpus/15.key',
 'corpus/16.key',
 'corpus/17.key',
 'corpus/18.key',
 'corpus/19.key',
 'corpus/2.key',
 'corpus/20.key',
 'corpus/21.key',
 'corpus/22.key',
 'corpus/23.key',
 'corpus/24.key',
 'corpus/25.key',
 'corpus/26.key',
 'corpus/27.key',
 'corpus/28.key',
 'corpus/29.key',
 'corpus/3.key',
 'corpus/30.key',
 'corpus/4.key',
 'corpus/6.key',
 'corpus/7.key',
 'corpus/8.key',
 'corpus/9.key']

In [212]:
def read(txt_files, sep = '$'): # функция препроцессинга (для вызова модуля из пакета), на выходе - список строк после препроцессинга
    preproc = []
    read = []
    for file in tqdm(txt_files):
        with open(file) as f:
            readd =  ''.join(f.read().split('\t'))
            
            read.append(readd.replace('\n', sep))
            
    return read

In [213]:
read_txt = read(txt_files, sep = ' ')

  0%|          | 0/27 [00:00<?, ?it/s]

In [214]:
read_txt[0]

'Does social capital determine innovation? To what extent? This paper deals with two questions: Does social capital determine innovation in manufacturing firms? If it is the case, to what extent? To deal with these questions, we review the literature on innovation in order to see how social capital came to be added to the other forms of capital as an explanatory variable of innovation. In doing so, we have been led to follow the dominating view of the literature on social capital and innovation which claims that social capital cannot be captured through a single indicator, but that it actually takes many different forms that must be accounted for. Therefore, to the traditional explanatory variables of innovation, we have added five forms of structural social capital (business network assets, information network assets, research network assets, participation assets, and relational assets) and one form of cognitive social capital (reciprocal trust). In a context where empirical investiga

In [215]:
read_key = read(key_files)

  0%|          | 0/27 [00:00<?, ?it/s]

In [216]:
read_key[0]

'innovation$manufacturing firms$business network assets$information network assets$research network assets$participation assets$relational assets$cognitive social capital$structural social capital$reciprocal trust$two-stage decision-making process$degree of radicalness$corporate modelling$decision theory$manufacturing industries$technological forecasting$$##$$social capital$innovation$questions$network assets$'

In [222]:
read_key_new = [el.split('#') for el in read_key]

In [223]:
read_key_new[0]

['innovation$manufacturing firms$business network assets$information network assets$research network assets$participation assets$relational assets$cognitive social capital$structural social capital$reciprocal trust$two-stage decision-making process$degree of radicalness$corporate modelling$decision theory$manufacturing industries$technological forecasting$$',
 '',
 '$$social capital$innovation$questions$network assets$']

In [338]:
read_key_act = []
read_key_my = []
read_key_true = []
for read_k in read_key_new:
    act = read_k[0].split('$')
    my = read_k[-1].split('$')
    intt = set(act) & set(my)

    read_key_act.append(act)
    read_key_my.append(my)
    read_key_true.append(intt)

In [341]:
 read_key_my[1]

['',
 '',
 'National learning systems',
 'technological change',
 'evidences',
 'late industrializing economies',
 'macroevidences',
 'Brazil',
 'South Korea',
 'processes',
 '']

In [342]:
 read_key_act[1]

['national learning systems',
 'technological change',
 'late industrializing economies',
 'Brazil',
 'South Korea',
 'national innovation system',
 'economic cybernetics',
 'technological forecasting',
 '',
 '']

In [343]:
read_key_true[1]

{'',
 'Brazil',
 'South Korea',
 'late industrializing economies',
 'technological change'}

Кол-во токенов в корпусе

In [237]:
lenn = 0
for elem in read_txt:
    lenn += len(elem.split(' '))
print(lenn)

3362


## 3.0 Лемматизация и токенизация

In [346]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag).lower() for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

# Lemmatize
text_lemm = []
key_lemm_my = []
key_lemm = []
key_lemm_intt = []
for i in tqdm(range(len(read_txt))):
    new_lem = []
    new_lem2 = []
    text_lemm.append(lemmatize_with_postag(read_txt[i]))
    for el in read_key_act[i]:
        new_lem.append(lemmatize_with_postag(el))
    for el2 in read_key_my[i]:
        new_lem2.append(lemmatize_with_postag(el2))
    new_lemm_intt = set(new_lem) & set(new_lem2)
    key_lemm.append(new_lem)
    key_lemm_my.append(new_lem2)
    key_lemm_intt.append(new_lemm_intt)

  0%|          | 0/27 [00:00<?, ?it/s]

In [244]:
text_lemm[0]

'does social capital determine innovation to what extent this paper deal with two question does social capital determine innovation in manufacturing firm if it be the case to what extent to deal with these question we review the literature on innovation in order to see how social capital come to be add to the other form of capital a an explanatory variable of innovation in do so we have be lead to follow the dominate view of the literature on social capital and innovation which claim that social capital can not be capture through a single indicator but that it actually take many different form that must be account for therefore to the traditional explanatory variable of innovation we have add five form of structural social capital business network asset information network asset research network asset participation asset and relational asset and one form of cognitive social capital reciprocal trust in a context where empirical investigation regard the relation between social capital an

In [245]:
key_lemm[0]

['innovation',
 'manufacturing firm',
 'business network asset',
 'information network asset',
 'research network asset',
 'participation asset',
 'relational asset',
 'cognitive social capital',
 'structural social capital',
 'reciprocal trust',
 'two-stage decision-making process',
 'degree of radicalness',
 'corporate modelling',
 'decision theory',
 'manufacturing industry',
 'technological forecasting',
 '',
 '']

In [246]:
key_lemm_my[0]

['', '', 'social capital', 'innovation', 'question', 'network asset', '']

In [348]:
key_lemm_intt[1]

{'',
 'brazil',
 'late industrializing economy',
 'national learning system',
 'south korea',
 'technological change'}

In [247]:
lenn = 0
for key in key_lemm:
    for elem in key:
        if len(elem.split())>lenn:
            lenn = len(elem.split())

In [248]:
lenn

5

## 3. Извлечение ключевых слов

#### RAKE

In [135]:
rake = RAKE.Rake(stop)

In [301]:
rake_list = []
for t in text_lemm:
  
    run=[elem[0] for elem in rake.run(t, maxWords=5, minFrequency=2)]
    rake_list.append(run)

In [302]:
rake_list[0]

['social capital determine innovation',
 'social capital',
 'innovation',
 'extent',
 'literature']

In [176]:
!pip install rake-nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [177]:
from rake_nltk import Rake
rake_nltk_var = Rake()

In [303]:
rake_nltk_list = []
for t in read_txt:
    rake_nltk_var.extract_keywords_from_text(t)
    keyword_extracted = rake_nltk_var.get_ranked_phrases()[:5]
    rake_nltk_list.append(keyword_extracted )

In [304]:
rake_nltk_list[0]

['actually takes many different forms',
 'providing new evidence regarding',
 'two decisionmaking stages considered',
 'social capital determine innovation',
 'social capital determine innovation']

возьмем это тоже

#### Text Rank

In [313]:
text_rank_list = []
text_rank_list_new = []
for t in text_lemm:
    run = [elem[0] for elem in keywords.keywords(t, language='english', additional_stopwords=stop, scores=True)]
    text_rank_list.append(run)

In [314]:
text_rank_list[0]

['social capital', 'network asset', 'explanatory', 'stage', 'paper', 'make']

In [315]:
text_rank_list_new = []
for t in read_txt:
    run = [elem[0] for elem in keywords.keywords(t, language='english', additional_stopwords=stop, scores=True)]
    text_rank_list_new.append(run)

In [316]:
text_rank_list_new[0]

['assets',
 'social capital',
 'network',
 'new',
 'stages',
 'explanatory',
 'makes']

#### YAKE!

Чем меньше скор, тем лучше

In [194]:
!pip install git+https://github.com/LIAAD/yake
import yake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-8_pmij7_
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-8_pmij7_
     |████████████████████████████████| 132 kB 7.6 MB/s 
  Created wheel for yake: filename=yake-0.4.8-py2.py3-none-any.whl size=62599 sha256=99f0e22092117000ca0939cde55387ede8a33b63305ff2a1afd7b4e3e8f0fddb
  Stored in directory: /tmp/pip-ephem-wheel-cache-6dq64gj5/wheels/52/79/f4/dae9309f60266aa3767a4381405002b6f2955fbcf038d804da
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73983 sha256=b1c7cbbfec30fb0a207ed127410a8df02ec368997d67b2c90fc3ded0521b7554
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built yake jellyfish


In [317]:
yake_li = []
kw_extractor = yake.KeywordExtractor(top=5, stopwords=stop)
for t in text_lemm:
    keywords = [el[0] for el in kw_extractor.extract_keywords(t)]
    yake_li.append(keywords)


In [318]:
yake_li[0]

['social capital determine',
 'capital determine innovation',
 'structural social capital',
 'cognitive social capital',
 'social capital come']

In [319]:
yake_li_new=[]
for t in read_txt:
    keywords = kw_extractor.extract_keywords(t)
    yake_li_new.append(keywords)

In [320]:
yake_li_new[0]

[('social capital', 0.0210969612352439),
 ('capital', 0.03954309252718451),
 ('social capital determine', 0.041654726023696485),
 ('social', 0.044838264177201305),
 ('capital determine innovation', 0.04927598777265478)]

Без лемматизации значения хуже, поэтому оставляем ее

## 4. Морфологические/синтаксические шаблоны

In [321]:
def find_pos(word):
    return nltk.pos_tag(word)

In [322]:
def filter_keywords(keywords):
    patterns = ['NOUN NOUN', 'ADJ NOUN', 'NOUN ADP NOUN', "NOUN", 'PROPN PROPN']
    pos_tags = []
    for i in keywords:
        filtered = []
        for key in i:
  
            doc = nlp(key)
            pos = ' '.join([word.pos_ for word in doc])
      
            if pos in patterns:
                filtered.append(key)
        pos_tags.append(filtered)
    return pos_tags


In [323]:
filt_rake = filter_keywords(rake_list)
filt_rake_nltk = filter_keywords(rake_nltk_list)
filt_rank = filter_keywords(text_rank_list)
filt_rank_nolem = filter_keywords(text_rank_list_new)
filt_yake = filter_keywords(yake_li)

## Оценить точность, полноту, F-меру

In [289]:
mlb = MultiLabelBinarizer()

In [329]:
def count_metrics(true, pred, filtered = 0):
    pred = mlb.transform(pred)
    f1 =  f1_score(true, pred, average='micro')
    prec = precision_score(true,pred, average='micro')
    rec = recall_score(true, pred, average='micro')
    return f1, prec, rec

In [355]:
true_keys = list(map(list,key_lemm_intt))

In [385]:
all_tags = true_keys + rake_list + text_rank_list + yake_li + rake_nltk_list + text_rank_list_new
mlb.fit(all_tags)
true = mlb.transform(true_keys)

In [386]:
count_metrics(true, rake_nltk_list)

(0.01639344262295082, 0.015625, 0.017241379310344827)

In [361]:
count_metrics(true, rake_list)

(0.11235955056179776, 0.16129032258064516, 0.08620689655172414)

In [388]:
import pandas as pd

filtered = [filt_rake, filt_rake_nltk, filt_rank, filt_rank_nolem, filt_yake]
index = ['F1-score', 'Precision', 'Recall']
columns = ['rake', 'rake_nltk', 'rank', 'rank_nolem', 'yake']
df = pd.DataFrame( index = index,
               columns =columns)
filt_vals = []
print("FILTERED")
def show_metrics(filtered):
    for i  in range(len(filtered)):
        f1, prec, rec = count_metrics(true, filtered[i])
        print(columns[i], ':')
        print('F1-score:', f1)
        print('Precision:', prec)
        print('Recall:', rec)

unfilt = [rake_list, rake_nltk_list, text_rank_list, text_rank_list_new, yake_li]
show_metrics(filtered)
print('-'*30)
print('NOT FILTERED')
show_metrics(unfilt)



FILTERED
rake :
F1-score: 0.06756756756756757
Precision: 0.15625
Recall: 0.04310344827586207
rake_nltk :
F1-score: 0.0
Precision: 0.0
Recall: 0.0
rank :
F1-score: 0.04278074866310161
Precision: 0.056338028169014086
Recall: 0.034482758620689655
rank_nolem :
F1-score: 0.01015228426395939
Precision: 0.012345679012345678
Recall: 0.008620689655172414
yake :
F1-score: 0.017094017094017096
Precision: 1.0
Recall: 0.008620689655172414
------------------------------
NOT FILTERED
rake :
F1-score: 0.11235955056179776
Precision: 0.16129032258064516
Recall: 0.08620689655172414
rake_nltk :
F1-score: 0.01639344262295082
Precision: 0.015625
Recall: 0.017241379310344827
rank :
F1-score: 0.03861003861003861
Precision: 0.03496503496503497
Recall: 0.04310344827586207
rank_nolem :
F1-score: 0.007168458781362007
Precision: 0.006134969325153374
Recall: 0.008620689655172414
yake :
F1-score: 0.0796812749003984
Precision: 0.07407407407407407
Recall: 0.08620689655172414


##  ошибки автоматического выделения ключевых слов

Можно заметить, что все не очень хорошо справились. Лучше всего был для не отфильтрованный rake, для отфильтрованных данных rank и rake относительно неплохие. 